In [ ]:
# @title Mental Health Text Classifier - Simplified Implementation

# This simplified version works with older versions of the transformers library
# It has all the essential functionality while avoiding compatibility issues

import torch
import pandas as pd
import numpy as np
import json
import os
from sklearn.model_selection import train_test_split
from transformers import GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from IPython.display import display, HTML
import matplotlib.pyplot as plt

print("Loading necessary libraries...")

# ==================== PART 1: DATA LOADING ====================

# Load the dataset
try:
    with open('/content/mental.json', 'r') as f:
        data = json.load(f)
    print("✅ Found mental.json file!")
except FileNotFoundError:
    print("⚠️ mental.json not found. Please upload it.")
    from google.colab import files
    uploaded = files.upload()
    with open(list(uploaded.keys())[0], 'r') as f:
        data = json.load(f)
    # Rename the file if needed
    if list(uploaded.keys())[0] != 'mental.json':
        os.rename(list(uploaded.keys())[0], 'mental.json')

# Convert to DataFrame
df = pd.DataFrame(data)
print(f"Dataset loaded with {len(df)} entries")

# Display distribution of labels
label_counts = df['label'].value_counts()
print("\nCategory distribution:")
print(label_counts)

# Extract unique labels and their suggestions
unique_labels = df['label'].unique()
label_dict = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label_dict.items()}

suggestion_dict = {}
for _, row in df.iterrows():
    suggestion_dict[row['label']] = row['suggestion']

print(f"Labels: {label_dict}")

# ==================== PART 2: MODEL TRAINING ====================

# Ask if the user wants to train the model
print("\nDo you want to train the model? This will take a few minutes.")
train_choice = input("Type 'yes' to train or 'no' to skip training (if model already exists): ").lower()

if train_choice == 'yes' or not os.path.exists('./mental_health_model'):
    print("\n🚀 Starting model training...")

    # Set up GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Convert labels to integers
    df['label_id'] = df['label'].map(label_dict)

    # Split data
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        df['text'].tolist(),
        df['label_id'].tolist(),
        test_size=0.2,
        random_state=42
    )

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    tokenizer.pad_token = tokenizer.eos_token

    # Load model with specified number of labels
    model = GPT2ForSequenceClassification.from_pretrained(
        'gpt2',
        num_labels=len(unique_labels)
    )
    model.config.pad_token_id = tokenizer.eos_token_id
    model = model.to(device)

    # Create a custom training loop instead of using Trainer
    from torch.utils.data import DataLoader, Dataset

    class MentalHealthDataset(Dataset):
        def __init__(self, texts, labels, tokenizer, max_length=128):
            self.encodings = tokenizer(texts, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')
            self.labels = torch.tensor(labels)

        def __getitem__(self, idx):
            item = {key: val[idx] for key, val in self.encodings.items()}
            item['labels'] = self.labels[idx]
            return item

        def __len__(self):
            return len(self.labels)

    # Create datasets
    train_dataset = MentalHealthDataset(train_texts, train_labels, tokenizer)
    val_dataset = MentalHealthDataset(val_texts, val_labels, tokenizer)

    # Create dataloaders
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8)

    # Set up optimizer
    optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

    # Training loop
    num_epochs = 10
    best_accuracy = 0

    print("Training progress:")
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        avg_train_loss = train_loss / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        predictions = []
        true_labels = []

        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

                logits = outputs.logits
                pred = torch.argmax(logits, dim=1).cpu().numpy()
                predictions.extend(pred)
                true_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        avg_val_loss = val_loss / len(val_loader)
        accuracy = accuracy_score(true_labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(true_labels, predictions, average='weighted')

        print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {avg_train_loss:.4f} - Val loss: {avg_val_loss:.4f} - Accuracy: {accuracy:.4f} - F1: {f1:.4f}")

        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            # Save model
            model_path = "./mental_health_model"
            os.makedirs(model_path, exist_ok=True)
            model.save_pretrained(model_path)
            tokenizer.save_pretrained(model_path)

            # Save label mappings and suggestions
            with open(os.path.join(model_path, 'id2label.json'), 'w') as f:
                json.dump({str(k): v for k, v in id2label.items()}, f)

            with open(os.path.join(model_path, 'suggestions.json'), 'w') as f:
                json.dump(suggestion_dict, f)

            print(f"✓ Saved better model with accuracy: {accuracy:.4f}")

    print("\n✅ Training complete!")
    print(f"Best model saved with accuracy: {best_accuracy:.4f}")

else:
    print("Skipping training, will try to use existing model...")

# ==================== PART 3: USER TESTING INTERFACE ====================

print("\n🔍 Setting up the testing interface...")

try:
    # Load model and tokenizer
    model_path = "./mental_health_model"
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2ForSequenceClassification.from_pretrained(model_path)

    # Load label mappings and suggestions
    with open(os.path.join(model_path, 'id2label.json'), 'r') as f:
        id2label = json.load(f)
        id2label = {int(k): v for k, v in id2label.items()}

    with open(os.path.join(model_path, 'suggestions.json'), 'r') as f:
        suggestion_dict = json.load(f)

    # Put model on GPU if available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    print("✅ Model loaded successfully!")

    # Function to predict sentiment
    def predict_sentiment(text):
        # Tokenize
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Get prediction
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

        # Get predicted class
        predicted_class_id = logits.argmax(-1).item()
        predicted_label = id2label[predicted_class_id]

        # Get confidence
        probabilities = torch.softmax(logits, dim=1)
        confidence = probabilities[0][predicted_class_id].item()

        # Get suggestion
        suggestion = suggestion_dict[predicted_label]

        return {
            "category": predicted_label,
            "confidence": confidence,
            "suggestion": suggestion
        }

    # Create a simple text-based interface
    print("\n==== Mental Health Text Classifier ====")
    print("Type your thoughts below and press Enter to analyze.")
    print("Type 'quit' to exit.")
    print("======================================")

    while True:
        # Get user input
        user_input = input("\nYour thoughts: ")

        if user_input.lower() == 'quit':
            print("Goodbye!")
            break

        if not user_input.strip():
            print("Please enter some text.")
            continue

        # Get prediction
        print("Analyzing...")
        result = predict_sentiment(user_input)

        # Format category name
        category_display = result['category'].replace('_', ' ').title()

        # Display result
        print("\n----- Analysis Result -----")
        print(f"Category: {category_display}")
        print(f"Confidence: {result['confidence']*100:.1f}%")
        print(f"Suggestion: {result['suggestion']}")
        print("--------------------------")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please make sure you've trained the model first.")import ipywidgets as widgets


Loading necessary libraries...
⚠️ mental.json not found. Please upload it.


Saving mental.json to mental.json
Dataset loaded with 49 entries

Category distribution:
label
depression        17
stress            16
emotional_load    16
Name: count, dtype: int64
Labels: {'depression': 0, 'stress': 1, 'emotional_load': 2}

Do you want to train the model? This will take a few minutes.
Type 'yes' to train or 'no' to skip training (if model already exists): yes

🚀 Starting model training...
Using device: cuda


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

...

Training progress:


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 1/10 - Train loss: 1.5376 - Val loss: 1.1183 - Accuracy: 0.2000 - F1: 0.0667
✓ Saved better model with accuracy: 0.2000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch 2/10 - Train loss: 1.0686 - Val loss: 1.0918 - Accuracy: 0.5000 - F1: 0.3810
✓ Saved better model with accuracy: 0.5000
Epoch 3/10 - Train loss: 1.0185 - Val loss: 0.7729 - Accuracy: 0.9000 - F1: 0.8984
✓ Saved better model with accuracy: 0.9000
Epoch 4/10 - Train loss: 0.9689 - Val loss: 0.7159 - Accuracy: 0.9000 - F1: 0.8984
Epoch 5/10 - Train loss: 0.7601 - Val loss: 0.6783 - Accuracy: 1.0000 - F1: 1.0000
✓ Saved better model with accuracy: 1.0000
Epoch 6/10 - Train loss: 0.6622 - Val loss: 0.5931 - Accuracy: 0.8000 - F1: 0.7867
Epoch 7/10 - Train loss: 0.3586 - Val loss: 0.4266 - Accuracy: 0.8000 - F1: 0.7867
Epoch 8/10 - Train loss: 0.2113 - Val loss: 0.1906 - Accuracy: 1.0000 - F1: 1.0000
Epoch 9/10 - Train loss: 0.0988 - Val loss: 0.1644 - Accuracy: 1.0000 - F1: 1.0000
Epoch 10/10 - Train loss: 0.0692 - Val loss: 0.1182 - Accuracy: 1.0000 - F1: 1.0000

✅ Training complete!
Best model saved with accuracy: 1.0000

🔍 Setting up the testing interface...
✅ Model loaded successf